In [26]:
import tensorflow as tf
import numpy as np
import os
import time

# Download the Shakespeare dataset

In [27]:
path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt' )

# Read the file

In [28]:
# read, then decode for py2 compat
text = open(path, 'rb').read().decode(encoding='utf-8')

In [29]:
#length of the text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [30]:
# examin the first 250 characters
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [31]:
# unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


# Process the text

<h3>Vectrize the text </h3>

Before training, we need to convert hte string to a numerical representation

In [32]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [33]:
#now create the tf.keras.layers.stringLookup

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary = list(vocab), mask_token=None
)

In [34]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use tf.keras.layers.StringLookup(..., invert=True).

In [35]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [36]:
# This layer recovers the characters from the vectors of IDs, and returns them as a tf.RaggedTensor of character
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [37]:
# we can tf.strings.reduce_join to join the characters back into strings.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [38]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

# The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

# Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [39]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [40]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [41]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [42]:
seq_length = 100


In [43]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder = True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [44]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [45]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [46]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [47]:
dataset = sequences.map(split_input_target)

In [48]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [49]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Create training batches
we used tf.data to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.


In [50]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000 # buffer size to shuffle the dataset

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Build the model
This section defines the model as a keras.Model subclass (For details see Making new Layers and Models via subclassing).

This model has three layers:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [51]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [52]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [53]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Try the model


In [54]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [55]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [56]:
# try it for the first example in the batch
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([36, 31, 53, 35, 49, 40, 28, 30, 26,  1, 62, 59,  7, 49,  7, 40, 42,
       65, 56, 17, 27, 62,  4,  8,  2,  5,  2, 26,  3,  9, 38, 12, 35, 29,
       45, 35, 17, 37, 33, 37,  3, 36, 54,  0, 41, 20, 61, 22, 60, 54, 63,
       31,  3, 26, 17, 20, 60, 49, 54,  4, 40, 54,  2, 21, 64, 31, 11, 55,
       25, 10, 24, 13, 17, 25, 47, 41, 27, 10, 22, 59, 54, 53, 11, 39, 41,
        1, 39, 30, 42,  4, 19, 35, 29, 57, 26, 51, 13, 27, 13, 33])

In [57]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'RICHARD:\nClifford, ask mercy and obtain no grace.\n\nEDWARD:\nClifford, repent in bootless penitence.\n\n'

Next Char Predictions:
 b'WRnVjaOQM\nwt,j,aczqDNw$- & M!.Y;VPfVDXTX!Wo[UNK]bGvIuoxR!MDGujo$ao HyR:pL3K?DLhbN3Iton:Zb\nZQc$FVPrMl?N?T'


# Train the model

In [58]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [59]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1879344, shape=(), dtype=float32)


In [60]:
tf.exp(example_batch_mean_loss).numpy()

65.88655

In [61]:
model.compile(optimizer='adam', loss=loss)

In [62]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [63]:
# execute the training
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 16s 57ms/step - loss: 2.6921
Epoch 2/20
172/172 [==============================] - 12s 56ms/step - loss: 1.9735
Epoch 3/20
172/172 [==============================] - 11s 53ms/step - loss: 1.6934
Epoch 4/20
172/172 [==============================] - 11s 52ms/step - loss: 1.5344
Epoch 5/20
172/172 [==============================] - 11s 52ms/step - loss: 1.4383
Epoch 6/20
172/172 [==============================] - 11s 52ms/step - loss: 1.3712
Epoch 7/20
172/172 [==============================] - 11s 52ms/step - loss: 1.3184
Epoch 8/20
172/172 [==============================] - 11s 55ms/step - loss: 1.2733
Epoch 9/20
172/172 [==============================] - 11s 53ms/step - loss: 1.2322
Epoch 10/20
172/172 [==============================] - 12s 53ms/step - loss: 1.1917
Epoch 11/20
172/172 [==============================] - 11s 52ms/step - loss: 1.1498
Epoch 12/20
172/172 [==============================] - 11s 53ms/step - loss: 1.1071
E

In [64]:
# Generate text
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [65]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [66]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Ah, my church ange; no, and on my knee must blame.
See that my daughter had betray my care return.

KING RICHARD III:
Sirrah young Venow, sworn begins to make
His body likeness to a most gaod.

MOMNA:
I have got him that walls, and durst not nurse?

Nark:
My Lord Fitzis a good suitor,
Even thought upon him.

All Those that are you!
Methinks that hand on seem his birthres like cords spriges;
My dagger is well slay-mings, all
his lords,
Shall fill'd in Henry lives, they come not,
Lood Katharina with them above,
And change our neck of his great grief,
With all his lordship trunked long,
Nor citizels not to London England's royal king;
Lest blowing betwixt his blood and sweet silver
Coring up: he seat of friends!
But let your restlusings that the time
'Your injuries are both beginness.

SOMERSET:
Say, Sbaching for your loves, untie me study
In whom a part of what it bodes; and
therefore I'll play my pash all I know;
And much I betray his very nor herself,
For I toward the nobles sho

In [67]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nI spy king and damnabling hoped!\nOf much I am too out. If it be sound,\nOr hollow princess; but that you own to speak upon his\nmealous. Hark you, he loves a bear.\nDarry, and some recreant and married; mock degree?\nNo, but mistaking come again,--\nNurse, come; for I know,\nSince I turn my house these may execute,\nOr sleeples, the secrets it's ranishment,\nThat it may jest on me, and thy\nsupper-each our gates, nightly to his country:\nHe humbles there did neelf villains, that\nCame but emboation; on thy eye I pluck my language.\n\nHENRY BOLINGBROKE:\nYour will take ord and will not do me but speak some: the people\nDe'er through'd you at the outward kept?\n\nGET:\nHere comes the cause. why, lo, her nothing, all;\nbut I have ta'en thee perform'd, but cannot'st be\npalsed here, till he comes.\n\nFirst Senator:\nTrue; how is't! spoke have the king is detched?\nThou hast done so. Alack, an heir,\nBut fright out mercy, where art thou the day,\nSo foul the duke a fl